In [1]:
import sys
print(sys.version)

3.7.9 (default, Feb 19 2021, 20:27:40) 
[Clang 12.0.0 (clang-1200.0.32.29)]


In [2]:
import os, sqlite3, config, sys
import pandas as pd
import backtrader as bt
from report import Cerebro
from strategy_classes import CrossOver, OpeningRangeBreakout

In [8]:
stock_id = 9395
strategy = "crossover"
start_date = '2020-04-20'
end_date = '2020-07-20'
set_cash = 30000

In [4]:
conn = sqlite3.connect(config.DB_FILE)

In [14]:
def saveplots(cerebro, numfigs=1, iplot=True, start=None, end=None,
             width=16, height=9, dpi=300, tight=True, use=None, file_path = '', **kwargs):

        from backtrader import plot
        if cerebro.p.oldsync:
            plotter = plot.Plot_OldSync(**kwargs)
        else:
            plotter = plot.Plot(**kwargs)

        figs = []
        for stratlist in cerebro.runstrats:
            for si, strat in enumerate(stratlist):
                rfig = plotter.plot(strat, figid=si * 100,
                                    numfigs=numfigs, iplot=iplot,
                                    start=start, end=end, use=use)
                figs.append(rfig)

        for fig in figs:
            for f in fig:
                f.savefig(file_path, bbox_inches='tight')
        return figs

In [15]:
def backtest(stock_id, strategy, conn, start_date=None, end_date=None, \
             open_range=None, liquidate_time='15:00:00', set_cash=25000):
    
    print(f"== Testing {stock_id} ==")
    
    df = pd.read_sql("""
        SELECT datetime, open, high, low, close, volume
        FROM stock_price_minute
        WHERE stock_id = :stock_id
        AND strftime('%Y-%m-%d', datetime) >= :start_date
        AND strftime('%Y-%m-%d', datetime) <= :end_date
        ORDER BY datetime ASC
        LIMIT 10000
        """, conn, params={"stock_id":9395,"start_date":start_date, \
                           "end_date":end_date}, index_col='datetime', parse_dates=['datetime'])
    data = df.between_time('09:30:00', '16:00:00')
    
    # initialize Cerebro engine, extende with report method
    cerebro = Cerebro()
    cerebro.broker.setcash(set_cash)
    cerebro.addsizer(bt.sizers.PercentSizer, percents=95)
    
    # add data
    feed = bt.feeds.PandasData(dataname=df)
    cerebro.adddata(feed)
    
    if strategy == 'opening_range_breakout':
        cerebro.addstrategy(strategy=OpeningRangeBreakout)
    else:
        # add Golden Cross strategy
        params = (('fast', 50),('slow', 200))
        cerebro.addstrategy(strategy=CrossOver, **dict(params))
        
    cerebro.run()
    
    saveplots(cerebro, file_path = 'backtest_output.png')
    
    cerebro.report(memo='run_id',
               outputdir='/Users/kylespringfield/Dev/MoneyTree/backtest_reports')

In [16]:
backtest(stock_id, strategy, conn, start_date=start_date, end_date=end_date, \
        set_cash=set_cash)

== Testing 9395 ==
2020-04-20 - 10:20:00, *** MKT: BTC/USD BUY: 407.5731379270401
2020-04-20 - 12:47:00, *** MKT: BTC/USD SELL: 407.5731379270401
2020-04-20 - 18:41:00, *** MKT: BTC/USD BUY: 411.21544648468074
2020-04-20 - 20:43:00, *** MKT: BTC/USD SELL: 411.21544648468074
2020-04-20 - 23:08:00, *** MKT: BTC/USD BUY: 411.3156162524097
2020-04-20 - 23:09:00, *** MKT: BTC/USD SELL: 411.3156162524097
2020-04-21 - 08:36:00, *** MKT: BTC/USD BUY: 411.44925188095283
2020-04-21 - 10:11:00, *** MKT: BTC/USD SELL: 411.44925188095283
2020-04-21 - 13:32:00, *** MKT: BTC/USD BUY: 417.00904629093924
2020-04-21 - 16:15:00, *** MKT: BTC/USD SELL: 417.00904629093924
2020-04-21 - 17:45:00, *** MKT: BTC/USD BUY: 415.88171378345976
2020-04-22 - 07:10:00, *** MKT: BTC/USD SELL: 415.88171378345976
2020-04-22 - 08:59:00, *** MKT: BTC/USD BUY: 413.79949448436855
2020-04-22 - 11:12:00, *** MKT: BTC/USD SELL: 413.79949448436855
2020-04-22 - 11:59:00, *** MKT: BTC/USD BUY: 413.04781923305734
2020-04-22 - 16:36

### sandbox

In [ ]:
start_date = '2020-04-20'
end_date = '2020-04-22'

In [ ]:
conn = sqlite3.connect(config.DB_FILE)

In [ ]:
df = pd.read_sql("""
        SELECT datetime, open, high, low, close, volume
        FROM stock_price_minute
        WHERE stock_id = :stock_id
        AND strftime('%Y-%m-%d', datetime) >= :start_date
        AND strftime('%Y-%m-%d', datetime) <= :end_date
        ORDER BY datetime ASC
        LIMIT 10000
    """, conn, params={"stock_id":9395,"start_date":start_date, \
                       "end_date":end_date}, index_col='datetime', parse_dates=['datetime'])
df.head()

In [ ]:
df = df.between_time('09:30:00', '16:00:00')

In [ ]:
kpi = {# PnL
       'start_cash': self.get_startcash(),
       'rpl': rpl,
       'result_won_trades': trade_analysis.won.pnl.total,
       'result_lost_trades': trade_analysis.lost.pnl.total,
       'profit_factor': (-1 * trade_analysis.won.pnl.total / trade_analysis.lost.pnl.total),
       'rpl_per_trade': rpl / trades_closed,
       'total_return': 100 * total_return,
       'annual_return': (100 * (1 + total_return)**(365.25 / bt_period_days) - 100),
       'max_money_drawdown': drawdown['max']['moneydown'],
       'max_pct_drawdown': drawdown['max']['drawdown'],
       # trades
       'total_number_trades': total_number_trades,
       'trades_closed': trades_closed,
       'pct_winning': 100 * trade_analysis.won.total / trades_closed,
       'pct_losing': 100 * trade_analysis.lost.total / trades_closed,
       'avg_money_winning': trade_analysis.won.pnl.average,
       'avg_money_losing':  trade_analysis.lost.pnl.average,
       'best_winning_trade': trade_analysis.won.pnl.max,
       'worst_losing_trade': trade_analysis.lost.pnl.max,
       #  performance
       'sharpe_ratio': sharpe_ratio,
       'sqn_score': sqn_score,
       'sqn_human': self._sqn2rating(sqn_score)
       }